In [ ]:
import numpy as np
import pandas as pd


In [38]:
movies=pd.read_csv('/content/tmdb_5000_movies.csv.zip')
credit=pd.read_csv('/content/tmdb_5000_credits.csv (1).zip')

In [ ]:
movies.head()

In [ ]:
movies=movies.merge(credit,on='title')

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.info()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.iloc[0].genres
import ast
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
import ast
def convert3(obj):
  L=[]
  convert=0
  for i in ast.literal_eval(obj):
    if convert!=3:
        L.append(i['name'])
        convert+=1
    else:
      break
  return L

In [ ]:
movies['cast'].apply(convert3)

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
def fetch_chr(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      L.append(i['name'])
      break
  return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_chr)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x: x.split())

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" "," ") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" "," ") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" "," ") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" "," ") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [37]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

744

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Gandhi')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [35]:
pickle.dump(new_df.to_dict,open('movie_dict.pkl','wb'))